# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,missoula,47.0166,-113.8009,13.22,24,0,4.25,US,1694803639
1,1,waitangi,-43.9535,-176.5597,8.23,77,100,5.01,NZ,1694803640
2,2,kievka,50.2619,71.5514,12.47,65,65,6.49,KZ,1694803520
3,3,prieska,-29.6641,22.7474,13.98,46,55,15.88,ZA,1694803640
4,4,ilulissat,69.2167,-51.1000,4.01,45,0,5.75,GL,1694803640


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
import hvplot.pandas
import holoviews as hv
from holoviews import opts

# Display the map
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size='Humidity',
    color='City',
    alpha = 0.5
)

map_plot.opts(opts.Points(size=7, tools=['hover'], width=600, height=400))

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = (city_data_df['Max Temp'] >= 21) & (city_data_df['Max Temp'] <= 27) & \
                   (city_data_df['Wind Speed'] <= 4.5) & (city_data_df['Cloudiness'] == 0)

# Drop any rows with null values
ideal_cities_df = city_data_df[ideal_conditions].dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
53,53,jingmen,31.0336,112.2047,21.63,80,0,2.39,CN,1694803645
83,83,buckner,38.3837,-85.4400,26.12,44,0,3.00,US,1694803648
103,103,tsiombe,-25.3000,45.4833,22.26,72,0,3.00,MG,1694803651
140,140,betioky,-23.7167,44.3833,21.64,63,0,2.75,MG,1694803655
197,197,uruzgan,32.8333,66.0000,22.04,17,0,0.49,AF,1694803661
203,203,jamestown,42.0970,-79.2353,22.23,48,0,3.44,US,1694803662
328,328,richland,40.7834,-82.5499,21.22,59,0,3.00,US,1694803675
388,388,toliara,-23.3500,43.6667,22.85,77,0,0.49,MG,1694803682
418,418,medford,42.3265,-122.8756,24.45,25,0,0.00,US,1694803686
439,439,blackfoot,43.1905,-112.3450,23.73,20,0,4.29,US,1694803589


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:, "Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
53,jingmen,CN,31.0336,112.2047,80,
83,buckner,US,38.3837,-85.4400,44,
103,tsiombe,MG,-25.3000,45.4833,72,
140,betioky,MG,-23.7167,44.3833,63,
197,uruzgan,AF,32.8333,66.0000,17,
203,jamestown,US,42.0970,-79.2353,48,
328,richland,US,40.7834,-82.5499,59,
388,toliara,MG,-23.3500,43.6667,77,
418,medford,US,42.3265,-122.8756,25,
439,blackfoot,US,43.1905,-112.3450,20,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "type": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jingmen - nearest hotel: Harbour City Hotel
buckner - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
betioky - nearest hotel: Hotel Salema
uruzgan - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
richland - nearest hotel: Country Inn & Suites
toliara - nearest hotel: Ambary
medford - nearest hotel: Sunset Inn
blackfoot - nearest hotel: Best Western Blackfoot Inn
lizzanello - nearest hotel: Best Western Plus Leone di Messapia
brookings - nearest hotel: Quality Inn
qingquan - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
53,jingmen,CN,31.0336,112.2047,80,Harbour City Hotel
83,buckner,US,38.3837,-85.4400,44,No hotel found
103,tsiombe,MG,-25.3000,45.4833,72,No hotel found
140,betioky,MG,-23.7167,44.3833,63,Hotel Salema
197,uruzgan,AF,32.8333,66.0000,17,No hotel found
203,jamestown,US,42.0970,-79.2353,48,DoubleTree Jamestown
328,richland,US,40.7834,-82.5499,59,Country Inn & Suites
388,toliara,MG,-23.3500,43.6667,77,Ambary
418,medford,US,42.3265,-122.8756,25,Sunset Inn
439,blackfoot,US,43.1905,-112.3450,20,Best Western Blackfoot Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    color='City',
    size='Humidity',
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'],
    alpha = 0.5
)



# Display the map
map_plot.opts(opts.Points(size=7, tools=['hover'], width=600, height=400))

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)